# Exposure age calculation

## The concentration at the surface

Cosmogenic radionuclides accumulate at the Earth surface and can be modelled as a function of exposure duration and erosion rates. In this notebook, we will explore these interactions. 

First, run the cell below to set the working environment (no need to understand these lines of code). If Google Colab warns about the trustfullness of the code, click on `Run anyway`.

In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display
from IPython import get_ipython
import os, sys, platform, shutil

# Define root directory according to current environment
if 'google.colab' in str(get_ipython()):

    dir_root = "/content/cosmo-training"

    if os.path.exists(dir_root):
      shutil.rmtree(dir_root)

    !git clone https://github.com/franz825/cosmo-training

else:
    # Path to root directory (repository)
    dir_root = os.path.dirname(sys.path[0])

# Import custom functions for CRN computation
sys.path.append(dir_root)
from src.functions.get_parameters_values import get_parameters_values
from src.functions.compute_surface_concentration import compute_surface_concentration
from src.functions.compute_exposure_age import compute_exposure_age

The parameters used as inputs to model cosmogenic nuclides are defined by the function `parameters_values()`.

If needed, you can modify these values directly in the function `.py` file and re-run the notebook from the beginning.

In [2]:
# Get parameters values defined in the function.
p = get_parameters_values()
display(p)

{'atn': 160,
 'prod_rate': 4.25,
 'rho': 2.7,
 'half_life_Be10': 1387000.0,
 'L': 4.997456240519106e-07}

Cosmogenic nuclides are the products of interactions of primary and secondary cosmic-ray particles with atomic nuclei. In-situ produced <sup>10</sup>Be cosmogenic radionuclides preferentially accumulate in quartz minerals. The accumulation of cosmogenic radionuclides is basically a function of (1) the production rate, (2) the inheritance, (3) the depth, (4) the erosion rate and (5) the exposure age.

To compute the <sup>10</sup>Be surface concentration, the equation below is used (Dunai, 2010). We asssume that exposure time is clocking since the onset of the erosion of the surface, with no pre-exposure memory and therefore no inheritance.

$$ C = \frac{P}{\lambda + \frac{\epsilon \cdot \rho}{\Lambda}} \cdot \Bigg[1 - \exp\Bigg(-T_{exp} \cdot \Big(\lambda + \frac{\epsilon \cdot \rho}{\Lambda}\Big)\Bigg)\Bigg] $$

With:
- $\lambda$: Decay constant (1/yr)
- $P$: Production rate (at/g/yr)
- $\rho$: Bulk density (g/cm³)
- $\Lambda$: Attenuation length (g/cm²)
- $T_{exp}$: Exposure time (yr)
- $\epsilon:$ Erosion rate (cm/yr)

Let's compute the <sup>10</sup>Be concentration of a surface eroding at 0.0001 cm/yr (or 1 m/Ma) for 10,000 yr: 

In [3]:
# 10Be concentration at the surface (at/g)
erosion_rate = 0.0001
exposure_age = 10000

concentration = compute_surface_concentration(erosion_rate, exposure_age, p)

# Return the result
print("CRN concentration (at/g) after " + str(exposure_age) + " years, with an erosion rate of " + str(erosion_rate) + " cm/yr:")
print(concentration)

CRN concentration (at/g) after 10000 years, with an erosion rate of 0.0001 cm/yr:
42038.5805532912


Re-run the previous Python cell with other values of erosion rates and exposure ages to evaluate their impact on modelled <sup>10</sup>Be concentrations.

**QUESTIONS**

1. How does the CRN surface concentration evolve when **exposure age** increases ?

2. How does the CRN surface concentration evolve when **erosion rate** increases ?

## The saturation of the concentration

Let's now automatize the computation of surface concentrations for a set of values of interest for erosion rate (cm/yr) and exposure age (a). 

Below, you can modify the two vectors of values, to explore the relationship in more details. 

In [4]:
# Constant erosion rate of the surface (cm/yr)
erosion_rates = [0.000001, 0.0001, 0.001, 0.005, 0.01, 0.03]

# Vector containing the exposure ages (yr) to process
# exposure_ages = [1000, 2000, 5000, 10000, 20000, 30000, 50000, 100000, 200000, 300000, 400000, 500000, 1000000]
# # Consider using a vector defining values at equal interval in a log space, using logspace function from numpy library
exposure_ages = np.logspace(3, 6, 1000)

# Print values
print("Erosion rates (cm/yr): ")
print(erosion_rates)
print("Exposure ages (a): ")
print(exposure_ages)

Erosion rates (cm/yr): 
[1e-06, 0.0001, 0.001, 0.005, 0.01, 0.03]
Exposure ages (a): 
[   1000.            1006.93863148    1013.92540756    1020.96066231
    1028.04473209    1035.17795563    1042.36067398    1049.59323056
    1056.87597118    1064.20924406    1071.59339982    1079.02879152
    1086.51577465    1094.05470721    1101.64594963    1109.2898649
    1116.98681847    1124.73717836    1132.54131515    1140.39960197
    1148.31241454    1156.28013121    1164.30313292    1172.38180329
    1180.51652857    1188.70769771    1196.95570236    1205.26093687
    1213.62379834    1222.04468663    1230.52400436    1239.06215695
    1247.65955263    1256.31660247    1265.0337204     1273.81132319
    1282.64983053    1291.54966501    1300.51125217    1309.53502048
    1318.62140139    1327.77082936    1336.98374182    1346.2605793
    1355.60178533    1365.00780655    1374.47909268    1384.01609657
    1393.61927422    1403.28908479    1413.0259906     1422.83045721
    1432.70295341  

Run the code below to compute the <sup>10</sup>Be concentration using a loop over the erosion rate and exposure age values defined above. 

In [5]:
# Containers for 10Be concentrations results
crn_table = pd.DataFrame({"exposure_age": exposure_ages})
crn_plot = pd.DataFrame()

# Loop within erosion rate values
for i in range(len(erosion_rates)):

    # Get current erosion rate
    erosion_rate = erosion_rates[i]

    # Array to contain CRN concentration for each 
    crn_concentrations = np.zeros(len(exposure_ages))

    # Loop within exposure age values
    for i in range(len(exposure_ages)): 
        
        # Get current exposure age
        exposure_age = exposure_ages[i]

        # Compute 10Be concentration at the surface 
        Be_concentration = compute_surface_concentration(erosion_rate, exposure_age, p)

        # Divide 10Be concentration by 1000 (and store in array
        crn_concentrations[i] = Be_concentration

    # Convert array of concentrations in a named serie
    crn_concentrations = pd.Series(crn_concentrations, name = "crn_conc_erate_" + str(erosion_rate), dtype = 'Float64')
    # Include array of concentrations in a complete dataframe
    crn_df = pd.DataFrame({"exposure_age": exposure_ages, "crn_concentration": crn_concentrations, "erosion_rate": pd.Series(np.repeat(erosion_rate, len(exposure_ages)))})

    # Collect concentrations formatted for plots
    crn_plot = pd.concat([crn_plot, crn_df], axis = 0)
    # Collect concentrations into pandas dataframe for csv export
    crn_table = pd.concat([crn_table, crn_concentrations], axis = 1)

Results are stored in two objects: `crn_table` that can be exported to .csv file for external use, and `crn_plot` that will be used below to plot the final graph.

In [6]:
# Write results to csv file
crn_table.to_csv("exposure_ages.csv")

# Print the table containing 10Be concentration for each exposure age
display(crn_table)

,exposure_age,crn_conc_erate_1e-06,crn_conc_erate_0.0001,crn_conc_erate_0.001,crn_conc_erate_0.005,crn_conc_erate_0.01,crn_conc_erate_0.03
0,1000.000000,4248.90237,4245.35549,4213.291573,4074.637343,3909.804555,3334.161122
1,1006.938631,4278.37627,4274.780021,4242.271068,4101.718939,3934.687601,3351.895699
2,1013.925408,4308.054572,4304.408266,4271.448112,4128.972333,3959.713792,3369.690433
3,1020.960662,4337.938693,4334.241634,4300.824025,4156.398503,3984.883745,3387.545091
4,1028.044732,4368.030058,4364.281541,4330.400139,4183.998431,4010.198079,3405.459433
...,...,...,...,...,...,...,...
995,972720.319245,3249485.899461,1711612.476538,244607.886507,50073.787777,25110.820606,8386.7827
996,979469.667070,3266809.929662,1715004.458644,244607.887745,50073.787777,25110.820606,8386.7827
997,986265.846131,3284193.23521,1718369.753359,244607.888852,50073.787777,25110.820606,8386.7827
998,993109.181375,3301635.595451,1721708.227491,244607.889843,50073.787777,25110.820606,8386.7827


Erosion lower the measured cosmogenic nuclide concentrations in samples from eroding surfaces, by bringing material to the surface that had been previously (partially) shielded from cosmic rays. Consequently, depending on the erosion rate, 10<sup>4</sup> to 10<sup>6</sup> years are required, before constant nuclide concentrations reflect the steady state erosion rate, i.e. before secular equilibrium between production and decay/erosion is reached. 

From the previous table, we can build a graph showing the relation between exposure age and <sup>10</sup>Be surface concentration for different values of erosion rates. 

In [7]:
# Convert erosion rate to m/Ma and coerce as string. 
crn_plot["erosion_rate_mma"] = crn_plot["erosion_rate"] * 10000
crn_plot["erosion_rate_mma"] = crn_plot["erosion_rate_mma"].astype(str)
# Convert concentrations from at/g to 10^3 at/g.
crn_plot["crn_concentration_103"] = crn_plot["crn_concentration"] / 1000

# Plot 10Be concentrations for various erosion rates 
plot = px.scatter(crn_plot, x = "exposure_age", y = "crn_concentration_103", color = "erosion_rate_mma", labels = dict(exposure_age = "Exposure age (a)", 
crn_concentration_103 = "<sup>10</sup>Be concentration (10<sup>3</sup> at/g)", erosion_rate_mma = "Erosion rate (m /Ma)"), log_x = True, log_y = True, width=1000, height=800)
plot.show()

**QUESTIONS**

1. What exposure duration is required to reach the secular equilibrium between production and erosion/decay in a landscape eroding at 100 m/Ma ?

2. How does this value evolve with decreasing erosion rate ? 

## The exposure dating

In the northwestern Scotland, we sample a granite moraine block (density = 2.7) and measure a <sup>10</sup>Be concentration of 85.3 x 10<sup>3</sup> at/g. We consider a production rate at sea level high latitude (SLHL) of 4.25 at/g/yr and the <sup>10</sup>Be half live of 1.37 x 10<sup>6</sup> yr. 

Exposure age can be written as follows: 

$$ T_{exp} = - \frac{1}{\lambda + \frac{\rho \cdot \epsilon}{\Lambda}} \cdot ln\Bigg[ 1-\frac{C \cdot \big(\lambda + \frac{\rho \cdot \epsilon}{\Lambda}\big)}{P} \Bigg] $$

With:
- $\lambda$: Decay constant (1/yr)
- $P$: Production rate (at/g/yr)
- $\rho$: Bulk density (g/cm³)
- $\Lambda$: Attenuation length (g/cm²)
- $\epsilon:$ Erosion rate (cm/yr)
- $C$: Cosmogenic concentration (at/g)

**QUESTIONS**

1. Using the function `compute_exposure_duration` below, compute the exposure age of the moraine considering an erosion of 1 mm / kyr. What is the exposure age of the moraine ? 

2. Test the influence of erosion rate on the exposure age value. Try, for instance, erosion rates of 2, 5, 10, and 20 m/Myr. 

3. How is the exposure age evolving as a function of the erosion rate ? Why ?

In [8]:
# 10Be concentration measured in the sample (at/g)
surface_concentration = 85300
# Erosion rate (cm/yr)
erosion_rate = 0.0001

# Compute exposure age (yr)
exposure_age = compute_exposure_age(surface_concentration, erosion_rate, p)

print(exposure_age)

20524.46366103237
